In [4]:
import mdtraj as md
import pyemma as pm

from pathlib import Path
import os
import pickle
import pandas 
import numpy as np

from openmm import app
import openmm as mm
import openmm.unit as unit
import openmmtools

import openpathsampling as ops
import openpathsampling.engines.openmm as ops_openmm
from openpathsampling.engines.openmm.tools import ops_load_trajectory
from openpathsampling.engines.openmm import trajectory_from_mdtraj

In [14]:
# Paths
title = 'CLN_tps_test'
model_path = r'./data/CLN/'
md_path = r'./data/init_paths/CLN_md/'
ops_path = Path(f'./data/{title}')
ops_path.mkdir(parents=True, exist_ok=True)

# Parameters
f_scheme = 'ca'

In [6]:
# Read tica_mod, kmeans_mod, and msm

with open(os.path.join(model_path, 'msm_models'), 'rb') as f:
    models = pickle.load(f)
tica_mod = models['tica_mod']
kmeans_mod = models['kmeans_mod']
msm = models['msm']

In [17]:
# Prepare CVs

def tica_cv(snapshot, tica_mod, f_scheme, tica_k):
    import openpathsampling as ops
    import mdtraj as md
    
    traj = ops.engines.Trajectory([snapshot]).to_mdtraj()
    traj.remove_solvent(inplace=True)
    
    f_traj = md.compute_contacts(traj, scheme=f_scheme)[0]
    tica_traj = tica_mod.transform(f_traj)[0]
    return tica_traj[tica_k]

def pcca_cv(snapshot, tica_mod, kmeans_mod, msm, f_scheme, pcca_k):
    import openpathsampling as ops
    import mdtraj as md
    
    traj = ops.engines.Trajectory([snapshot]).to_mdtraj()
    traj.remove_solvent(inplace=True)
    
    f_traj = md.compute_contacts(traj, scheme=f_scheme)[0]
    tica_traj = tica_mod.transform(f_traj)
    microstate = kmeans_mod.assign(tica_traj)
    pcca = msm.metastable_memberships[microstate].flatten()
    return pcca[pcca_k]

with open(os.path.join(ops_path, 'cvs'), 'wb') as f:
    models = pickle.dump({'tica_cv':tica_cv, 'pcca_cv': pcca_cv},f)

tica = ops.FunctionCV("tica_cv", tica_cv,
                      tica_mod=tica_mod,
                      f_scheme=f_scheme,
                      tica_k=0)
pcca = ops.FunctionCV("pcca_cv", pcca_cv, 
                     tica_mod=tica_mod,
                     kmeans_mod=kmeans_mod,
                     msm=msm,
                     cv_time_reversible=True,
                     f_scheme=f_scheme,
                     pcca_k=0)

# Define metastable state hypervolumes
pcca_0_vol = ops.volume.CVDefinedVolume(pcca, lambda_min=0.95, lambda_max=1.00)
pcca_1_vol = ops.volume.CVDefinedVolume(pcca, lambda_min=0.00, lambda_max=0.05)

In [ ]:
gro = app.GromacsGroFile(os.path.join(md_path, 'nvt.gro'))
# Gmx topology only stores reference to ff definitions. Need to specifty the directory contains ff files.
top = app.GromacsTopFile(os.path.join(md_path, 'topol.top'), 
                         periodicBoxVectors=gro.getPeriodicBoxVectors(),
                         includeDir=r'/home/rzhu/Loc/gromacs/share/gromacs/top')

# Create system from gmx files
system = top.createSystem(nonbondedMethod=app.PME, 
                          nonbondedCutoff=1.0*unit.nanometers,
                          constraints=app.HBonds, 
                          rigidWater=True,
                          ewaldErrorTolerance=0.0005)

# Velocity-Verlet with Velocity Randomisation Integrator that simulates Langvein dynamics
integrator = openmmtools.integrators.VVVRIntegrator(
    # Temperature
    340*unit.kelvin,
    # Collision rate (temperature coupling time constant)
    1.0/unit.picoseconds,
    # Timestep
    2.0*unit.femtoseconds
)
integrator.setConstraintTolerance(0.00001)

In [ ]:
# Parallelisation may slow down simulation. It's dependent on the system size (the larger the better), connection speed between gpus (nv>>pcie), gpu power,
# and specific forces. 

openmm_properties = {'DeviceIndex':'2'}
platform = mm.openmm.Platform.getPlatformByName('CUDA')
engine_options = {
    # Maximal frames per trajectory 100000 *1 ps = 100 ns
    'n_frames_max': 100000,
    # Time interval between frames = 1 ps
    'n_steps_per_frame': 500
}
ops_topology = ops_openmm.tools.topology_from_pdb(os.path.join(md_path, 'nvt.gro'))
engine = ops_openmm.Engine(
    topology=ops_topology, 
    system=system, 
    integrator=integrator, 
    openmm_properties=openmm_properties,
    options=engine_options
).named('CLN_Openmm_engine')
engine.initialize(platform)

In [ ]:
# Define the path ensemble to be sampled from
network = ops.TPSNetwork(initial_states=pcca_0_vol, final_states=pcca_1_vol)
# Define shooting scheme. A shooting move scheme can contains multiple shooting strategies 
scheme = ops.OneWayShootingMoveScheme(network=network, 
                                      selector=ops.UniformSelector(),
                                      engine=engine).named("CLN_scheme")

In [ ]:
# Load initial trajectory from external files
init_traj=trajectory_from_mdtraj(md.load(os.path.join(md_path, 'trans.pdb')))
initial_conditions = scheme.initial_conditions_from_trajectories(init_traj)

In [ ]:
storage = ops.Storage(ops_path.join(f'{title}.nc'), "w", template=init_traj[0])
sampler = ops.PathSampling(storage=storage,
                           move_scheme=scheme,
                           sample_set=initial_conditions)

In [ ]:
sampler.run(50)

In [ ]:
storage.close()